In [ ]:
pip install transformers

In [ ]:
!pip install tensorflow

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import platform
import time
import pathlib
import os
import json
import zipfile
import pandas as pd

# from google.colab import files

print('Python version:', platform.python_version())
print('Tensorflow version:', tf.__version__)

Python version: 3.10.12
Tensorflow version: 2.13.0


In [ ]:
import os

current_directory = os.getcwd()
print("La tokenización se está guardando en la siguiente carpeta:", current_directory)

La tokenización se está guardando en la siguiente carpeta: /content


In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/TFM/Dataset Definitivo')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TFM/Dataset Definitivo/RAW_recipes.csv')
print(f'Número de filas: {df.shape[0]}, Número de columnas:{df.shape[1]}')

Número de filas: 231637, Número de columnas:12


In [ ]:
dataset_raw = df[:231637]
dataset_raw[['calories','total fat','sugar','sodium','protein','saturated fat','carbohydrates']] = dataset_raw.nutrition.str.split(",",expand=True)
dataset_raw['calories'] = dataset_raw['calories'].apply(lambda x: x.replace("[" ,""))
dataset_raw['carbohydrates'] = dataset_raw['carbohydrates'].apply(lambda x: x.replace("]" ,""))
dataset_raw[['calories','total fat','sugar','sodium','protein','saturated fat','carbohydrates']] =  dataset_raw[['calories','total fat','sugar','sodium','protein','saturated fat','carbohydrates']].astype(float)

<ipython-input-9-f5d0deb0fae5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_raw[['calories','total fat','sugar','sodium','protein','saturated fat','carbohydrates']] = dataset_raw.nutrition.str.split(",",expand=True)
<ipython-input-9-f5d0deb0fae5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_raw[['calories','total fat','sugar','sodium','protein','saturated fat','carbohydrates']] = dataset_raw.nutrition.str.split(",",expand=True)
<ipython-input-9-f5d0deb0fae5>:2: SettingWithCopyWarning

In [ ]:
dataset_raw

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories,total fat,sugar,sodium,protein,saturated fat,carbohydrates
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22,415.2,26.0,34.0,26.0,44.0,21.0,15.0
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13,14.8,0.0,2.0,58.0,1.0,0.0,1.0
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8,59.2,6.0,2.0,3.0,6.0,5.0,0.0
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10,188.0,11.0,57.0,11.0,7.0,21.0,9.0


In [ ]:
dataset_raw = dataset_raw.drop(["name", "id", "contributor_id", "submitted", "tags", "nutrition", "n_steps", "description", "n_ingredients", "total fat", "sugar", "sodium", "protein", "saturated fat", "carbohydrates"], axis=1)

In [ ]:
!pip install transformers[torch] -U
!pip install transformers[torch]
!pip install accelerate -U

In [ ]:
# Selecciona solo los primeros 10,000 registros
reduced_dataset_raw = dataset_raw.head(10000)


In [ ]:
from transformers import GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

# Define una función para combinar las columnas
def combine_columns(row):
    return f"Tiempo: {row['minutes']} minutos\nIngredientes: {', '.join(row['ingredients'])}\nPasos: {row['steps']}\nCalorías: {row['calories']}"

# Aplica la función a tu conjunto de datos reducido
reduced_dataset_raw["text"] = reduced_dataset_raw.apply(combine_columns, axis=1)

# Divide el conjunto de datos reducido en entrenamiento y validación
train_df, valid_df = train_test_split(reduced_dataset_raw, test_size=0.2, random_state=42)

# Obtén las listas de textos de entrenamiento y validación
train_texts = train_df["text"].tolist()
valid_texts = valid_df["text"].tolist()

# Inicializa el tokenizador GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Define una función para tokenizar textos
def tokenize_texts(texts, tokenizer, block_size):
    tokenized_texts = []
    for text in texts:
        tokens = tokenizer.encode(text, max_length=block_size, truncation=True)
        tokenized_texts.append(tokens)
    return tokenized_texts

# Tamaño del bloque (block size) para tokenización
block_size = 128

# Tokeniza los textos de entrenamiento y validación
train_tokenized_texts = tokenize_texts(train_texts, tokenizer, block_size)
valid_tokenized_texts = tokenize_texts(valid_texts, tokenizer, block_size)

# Define un conjunto de datos personalizado
class TokenizedTextDataset(Dataset):
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts)

    def __getitem__(self, idx):
        return torch.tensor(self.tokenized_texts[idx])

# Crea conjuntos de datos para entrenamiento y validación
train_dataset = TokenizedTextDataset(train_tokenized_texts)
valid_dataset = TokenizedTextDataset(valid_tokenized_texts)

# Crea un tokenizador GPT-2 personalizado con un token de relleno
pad_token = tokenizer.eos_token  # Puedes usar cualquier token que desees como token de relleno
tokenizer.pad_token = pad_token

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, pad_to_multiple_of=8
)

# Define los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=1,  # Reduce el número de épocas
    per_device_train_batch_size=8,  # Aumenta el tamaño del lote de entrenamiento
    per_device_eval_batch_size=8,
    eval_steps=10,
    save_steps=10,
    warmup_steps=10,
    prediction_loss_only=True,
)

# Crea el objeto Trainer y comienza el entrenamiento
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

trainer.train()

<ipython-input-14-b63638bfb883>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_dataset_raw["text"] = reduced_dataset_raw.apply(combine_columns, axis=1)


Step,Training Loss
500,0.787900
1000,0.516800


TrainOutput(global_step=1000, training_loss=0.6523706359863282, metrics={'train_runtime': 2047.2836, 'train_samples_per_second': 3.908, 'train_steps_per_second': 0.488, 'total_flos': 522584064000000.0, 'train_loss': 0.6523706359863282, 'epoch': 1.0})

Guardar el modelo

In [ ]:
# Monta Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define la ruta de guardado
save_path = '/content/drive/MyDrive/Colab Notebooks/TFM/GPT'

# Guarda el tokenizador
tokenizer.save_pretrained(save_path)

# Guarda el modelo
model.save_pretrained(save_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


para abrir

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Monta Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define la ruta de carga
load_path = '/content/drive/MyDrive/Colab Notebooks/TFM/GPT'

# Carga el tokenizador
tokenizer = GPT2Tokenizer.from_pretrained(load_path)

# Carga el modelo
model = GPT2LMHeadModel.from_pretrained(load_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import GPT2LMHeadModel

def generate_recipe(ingredients, model, tokenizer, max_length=100, num_return_sequences=1):
    # Combina los ingredientes en una cadena de texto
    ingredients_text = ', '.join(ingredients)

    # Crea un texto de entrada con los ingredientes
    input_text = f"Ingredientes: {ingredients_text}\nPasos:"

    # Codifica el texto de entrada
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Codifica el texto de entrada
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Crea una attention_mask
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

    # Genera texto con el modelo
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        temperature=0.4,
        top_k=50,
        top_p=0.95,
    )

    # Decodifica el texto generado
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text
# Define la ruta de guardado
save_path = '/content/drive/MyDrive/Colab Notebooks/TFM/GPT'

# Carga el modelo entrenado
model = GPT2LMHeadModel.from_pretrained(save_path)

# Ejemplo de ingredientes
ingredients = ["cheese", "salt", "eggs"]

# Genera una receta utilizando los ingredientes proporcionados
generated_recipe = generate_recipe(ingredients, model, tokenizer)
print(generated_recipe)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ingredientes: cheese, salt, eggs
Pasos: ['preheat oven to 350 degrees', 'pre-heat grill to 375 degrees, and then to 425 degrees for the cheese', "preferably with a large skillet", 'add the butter and stir until melted', "'add a little more butter if desired', add the eggs and cook for about 5 minutes', and'remove from heat', ]
Calorías: 215.8.1.6
S
